### Idiomatic Python: Custom Sorts and Leveraging Stable Sorts

We often need to sort a collection of objects based on some custom sort order. 

Python provides us with the highly efficient `sorted()` function (Timsort algorithm, based on both the merge sort and insertion sort algorithms - see [here](https://en.wikipedia.org/wiki/Timsort)) which can sort a collection, not only based on the natural ordering of the elements in the collection (assuming there is one), but also based on a value derived from each element. 

One of the really important aspects of the Timsort algorithm, is that it is a **stable** algorithm. 

What this means is that the relative order of two elements that are considered equal, will remain in the same relative order.

That's maybe a little difficult to digest at first, but a simple example will demonstrate this simple concept.

Suppose we have these tuples:

In [1]:
t1 = (1, 2, 3)
t2 = (2, 3, 4)
t3 = (1, 2, 3)
t4 = (2, 3, 4)

Now, let's make a list out of them:

In [2]:
l = [t1, t2, t3, t4]

Next, we will sort that list based on the natural ordering of these tuples:

In [3]:
s = sorted(l)

Let's see what that sorted list looks like:

In [4]:
s

[(1, 2, 3), (1, 2, 3), (2, 3, 4), (2, 3, 4)]

As you can see, the tuples are sorted correctly - but, more importantly here, let's look at the relative "position" of the equal tuples:

In [5]:
[id(t) for t in s]

[4389434752, 4394403328, 4394404416, 4394601408]

And compare that to the order of the tuples in the original unsorted list:

In [6]:
[id(t) for t in l]

[4389434752, 4394404416, 4394403328, 4394601408]

Notice how the relative order of `t1` and `t3` was maintained, as well as the relative order of `t2` and `t4`. This is what is meant by a stable sort.

The reason this is an important property of the Timsort is that it allows us to sort an iterable using multiple passes - we'll come back to that point later.

The `sorted` function allows us to specify a `key` function in the call to `sorted()`.

This key function allows us to assign a different value to each element in the iterable, and have the iterable sorted by that value (the **key value**) instead of the original element value (which may not even have a natural sort order).

The key function is simply a function that transforms each element in the iterable to another value we want to sort by - so it is **always** a function of **one** variable (each element will be passed to that function during the sort), and returns the value we want to sort by.

Let's look at a very simple example first.

Suppose we have this list:

In [7]:
l = [1, 2, -1, 3, -4, 5]

If we sort this list, we get this:

In [8]:
sorted(l)

[-4, -1, 1, 2, 3, 5]

But suppose we really want to sort by the magnitude of each element (it's absolute value), basically ignoring the negative signs.

This means we want to sort that list not by the original value, but by the absolute value of each element. So our key function will be a function that takes on argument (the value in the list), and returns the absilute value of that element.

We could create our own function to do that:

In [9]:
def absolute(x):
    if x < 0:
        return -x
    return x

Then we can sort using `abs` as our key function:

In [10]:
sorted(l, key=absolute)

[1, -1, 2, 3, -4, 5]

Of course, for the absolute value we really do not need to define our own function, since Python already provides us (a few!), such as the built-in `abs()` function:

In [11]:
sorted(l, key=abs)

[1, -1, 2, 3, -4, 5]

You can see that by default the sort order is **ascending** - we can easily reverse that, using the `reverse` argument of the `sorted` function:

In [12]:
sorted(l, key=abs, reverse=True)

[5, -4, 3, 2, 1, -1]

Now this `key` argument can be very powerful.

One module that is very helpful in this context is the `operator` module, docs [here](https://docs.python.org/3/library/operator.html#module-operator)

It provides us a functional approach to many operators. In particular we're going to make use of the `attrgetter` and `itemgetter` functions.

Let's look at `attrgetter` first.

In [13]:
from operator import attrgetter

Suppose we have a class such as this:

In [14]:
from math import dist

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    @property
    def quadrant(self):
        if self.x > 0:
            if self.y > 0:
                return 1
            return 4
        else:
            if self.y > 0:
                return 2
            return 3
        return 0
    
    def __abs__(self):
        return dist((0,0), (self.x, self.y))
    
    def __repr__(self):
        return f"Point({self.x}, {self.y}, magnitude={abs(self):.3f}, quadrant={self.quadrant})"

In [15]:
pt = Point(-1, -1)
pt

Point(-1, -1, magnitude=1.414, quadrant=3)

Now suppose we have the following tuple of points:

In [16]:
points = (
    Point(-5, 2), 
    Point(2, 2), 
    Point(-3, -1), 
    Point(2, -1),
    Point(-2, 2),
    Point(2, 5),
    Point(-2, -3)
)

Each of those points have a distance from the origin:

In [17]:
[abs(p) for p in points]

[5.385164807134504,
 2.8284271247461903,
 3.1622776601683795,
 2.23606797749979,
 2.8284271247461903,
 5.385164807134504,
 3.605551275463989]

And they also belong to some quadrant:

In [18]:
[p.quadrant for p in points]

[2, 1, 3, 4, 2, 1, 3]

Now, we already know how to sort by distance from the origin:

In [19]:
sorted(points, key=abs)

[Point(2, -1, magnitude=2.236, quadrant=4),
 Point(2, 2, magnitude=2.828, quadrant=1),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(2, 5, magnitude=5.385, quadrant=1)]

But what if we wanted to sort by quadrant instead?

We need to provide a `key` function that will return the quadrant for each point.

We can do this by defining our own function that will receive a point as the argument, and return the quadrant of the point:

In [20]:
sorted(points, key=lambda pt: pt.quadrant)

[Point(2, 2, magnitude=2.828, quadrant=1),
 Point(2, 5, magnitude=5.385, quadrant=1),
 Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(2, -1, magnitude=2.236, quadrant=4)]

But, even simpler than creating a lambda function is to use the `attrgetter` function. Basically this functions returns a new function that will get the named attribute of whatever object gets passed to it.

This is very similar to the `getattr` function:

In [21]:
p = Point(2, 2)
getattr(p, 'quadrant')

1

We can use the `attrgetter` function this way:

In [22]:
get_quadrant = attrgetter("quadrant")

Note that `quadrant` is a function, and can be called by passing a `Point` instance to it, and it will return the `quadrant` attribute of the point:

In [23]:
get_quadrant(p)

1

In essence, this is the same as writing:

In [24]:
lambda pt: pt.quadrant

<function __main__.<lambda>(pt)>

Let's do some quick timings to determine which retrieval is faster, using `attrgetter` vs that user defined function:

In [25]:
from timeit import timeit

In [26]:
p = Point(-2, 1)

func = attrgetter("quadrant")
result_1 = timeit("func(p)", globals=globals(), number=10_000_000)

func = lambda pt: pt.quadrant
result_2 = timeit("func(p)", globals=globals(), number=10_000_000)
print(f"{result_1:.3f}")
print(f"{result_2:.3f}")

0.936
1.111


As you can see, using `attrgetter` is not only more readable code, but also faster.

So now, we can use this more expressive approach to sorting our `points` tuple:

In [27]:
sorted(points, key=attrgetter("quadrant"))

[Point(2, 2, magnitude=2.828, quadrant=1),
 Point(2, 5, magnitude=5.385, quadrant=1),
 Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(2, -1, magnitude=2.236, quadrant=4)]

Now suppose we want to sort our `points` tuple this way: by quadrant ascending, and **then** by distance from origin descending.

Since Timsort is a stable sort, we could do it in two steps:

- first sort by origin (descending)
- then sort by quadrant (ascending)

Remember that when we sort by quadrant, points with equal quadrants will maintain their original relative positions, which was the distance to the origin (or magnitude).

In [28]:
sort_1 = sorted(points, key=abs, reverse=True)
sort_1

[Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(2, 5, magnitude=5.385, quadrant=1),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(2, 2, magnitude=2.828, quadrant=1),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(2, -1, magnitude=2.236, quadrant=4)]

In [29]:
final_sort = sorted(sort_1, key=attrgetter("quadrant"))
final_sort

[Point(2, 5, magnitude=5.385, quadrant=1),
 Point(2, 2, magnitude=2.828, quadrant=1),
 Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(2, -1, magnitude=2.236, quadrant=4)]

Of course, you could also do this in a single sort by using a more complex key function:

In [30]:
def sort_by_quad_mag(pt: Point):
    return pt.quadrant, -abs(pt)

In [31]:
sorted(points, key=sort_by_quad_mag)

[Point(2, 5, magnitude=5.385, quadrant=1),
 Point(2, 2, magnitude=2.828, quadrant=1),
 Point(-5, 2, magnitude=5.385, quadrant=2),
 Point(-2, 2, magnitude=2.828, quadrant=2),
 Point(-2, -3, magnitude=3.606, quadrant=3),
 Point(-3, -1, magnitude=3.162, quadrant=3),
 Point(2, -1, magnitude=2.236, quadrant=4)]

Now, you may be wondering the performance impact of sorting in two steps, vs using a more complex (and less readable) key function:

In [32]:
temp = sorted(points, key=abs, reverse=True)
final = sorted(sort_1, key=attrgetter("quadrant"))

vs

In [33]:
final = sorted(points, key=lambda pt: (pt.quadrant, -abs(pt)))

Well, that's a good question, let's find out using some timings.

In [34]:
from timeit import timeit
from random import randint, seed

First we create a list of `10_000` points:

In [35]:
seed(0)
points = [
    Point(randint(-1_000, 1_000), randint(-1_000, 1_000))
    for _ in range(10_000)
]

We're going to use these two variants of the sort:

In [36]:
sorted_1 = sorted(sorted(points, key=abs, reverse=True), key=attrgetter('quadrant'))
sorted_2 = sorted(points, key=sort_by_quad_mag)

And just to sanity check our work, let's make sure the results are the same:

In [37]:
sorted_1 == sorted_2

True

Ok, so now let's time them:

In [38]:
key_func = attrgetter("quadrant")
test_1 = timeit(
    "sorted(sorted(points, key=abs, reverse=True), key=key_func)",
    globals=globals(),
    number=1_000
)

key_func = lambda pt: (pt.quadrant, -abs(pt))
test_2 = timeit(
    "sorted(points, key=key_func)",
    globals=globals(),
    number=1_000
)

print(f"{test_1=:.3f}")
print(f"{test_2=:.3f}")

test_1=4.070
test_2=5.129


Surprised?? I was too! The only thing I could think of is that Timsort is very efficient if the data is already partially ordered - which it is, since we stacked one sort after another. If you know different, **please** let us know in the comments - I am quite curious.

However, in general do not rely on one approach always executing faster than the other - if the sorts are a potential bottleneck in your app, try both ways, and see which one performs better with **your** data.

Of course, there is also a memory tradeoff between the two approaches - in the "stacked" approach, we essentially create an extra list for each intermediate sort (one extra in this particular example), whereas the slower version only ends up with one single list - the final one.

The next function I wanted to look at in the `operator` module was the `itemgetter` function. Basically it is a function that returns another function that retrieves an item from a sequence type by index.

Let's take a quick look:

In [39]:
from operator import itemgetter
get_item_2 = itemgetter(2)

In [40]:
get_item_2("abcd")

'c'

As you can see it retrieved the character at index `2`.

So, we could sort an iterable of sequence types based on a specific element (by index) of each sequence:

In [41]:
l = [
    "ONI",
    "CGI",
    "INR",
    "CIA",
    "NSA",
    "DIA",
    "OIA",
    "PDQ"
]

Now suppose we want to sort based on the second character in each string.

We can use a custom key function using a lambda:

In [42]:
sorted(l, key=lambda s: s[1])

['PDQ', 'CGI', 'CIA', 'DIA', 'OIA', 'ONI', 'INR', 'NSA']

or we can use the `itemgetter` function:

In [43]:
sorted(l, key=itemgetter(1))

['PDQ', 'CGI', 'CIA', 'DIA', 'OIA', 'ONI', 'INR', 'NSA']

Something worth looking at is whether there is a performance difference between the two approaches:

In [44]:
key_func = lambda s: s[1]
timeit("sorted(l, key=key_func)", globals=globals(), number=10_000_000)

4.635904707945883

In [45]:
key_func = itemgetter(1)
timeit("sorted(l, key=key_func)", globals=globals(), number=10_000_000)

3.5795188338961452

So, like before, it looks like using `itemgetter` is faster than using a user defined function, and, in my opinion, is more readable code.

As usual, you have a number of options available to you when sorting - but before you decide on always using a specific pattern because you think it is more performant, take a minute to determine if the sort is going to be a bottleneck in your application (and make certain, don't guess!), then prefer readability over a negligible performance increase that will not affect your overall program's run time.